# M5.3 - Setting Up One-Time Data Processing

*Part of:* [**Open Climate Science for Crops & Crop Conditions**](https://github.com/OpenClimateScience/M5-Open-Science-for-Crops)

In [ ]:
import datetime
import glob
import os
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio
import rioxarray
import py4eos
import pyproj
from shapely.geometry import Polygon
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib import pyplot
from tqdm import tqdm

auth = earthaccess.login()

LC_DIR = 'data/MCD12Q1'
VNP16_DIR = 'data/VNP16A2GF'
IMERG_DIR = 'data/IMERG'
OUTPUT_LC_FILENAME = 'data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h15v05_2023.tiff'
OUTPUT_VNP16_DIR = 'data/processed/VNP16_ET_and_PET'
OUTPUT_IMERG_DIR = 'data/processed'
TIME_PERIOD = ('2023-10-01', '2024-09-30')

## Overview

To compute the water requirement satisfaction index (WRSI), we will need the following data:

- A land-cover map (to help us identify where croplands are in our study area)
- Potential evapotranspiration (PET)
- Precipitation

For each of these, NASA has a product we can use:

- [The MODIS MCD12Q1 dataset](https://doi.org/10.5067/MODIS/MCD12Q1.061) describes the land-cover class for each 500-m pixel on the MODIS sinusoidal grid.
- As we've seen previously, [the MODIS MOD16A2 product](https://doi.org/10.5067/MODIS/MOD16A2.006) provides terrestrial evapotranspiration (ET) and PET estimates every 8 days across the entire globe.
- And we've also previously used NASA's precipitation data from [the Global Precipitation Monitoring (GPM) mission's IMERG-Final dataset.](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDF_06/summary)

**In this lesson, we'll prepare the datasets we need for computing the WRSI. Importantly, we will also see how to write Snakemake rules that will enable future users to quickly reproduce our data processing steps.**

Specifically, we want to:

1. Resample and project the land-cover data onto a 1-km EASE-Grid 2.0, for easy comparison to other datasets.

---

## Getting the data we need

We'll continue to use the `earthaccess` library for programmatic access to NASA Earthdata Search. Recall that the `earthaccess.search_data()` function has a `bounding_box` argument that allows us to search for datasets that fall within a rectangular bounding box, defined by latitude-longitude coordinates.

```python
help(earthaccess.search_data)
```

```
**bounding_box**: a tuple representing spatial bounds in the form
    `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`
```

We'll define our bounding box, `bbox`, for our study area, a part of Northern Africa centered on Algeria.

In [ ]:
# The bounding box for our study area
bbox = (1.5, 34.0, 8.0, 37.0)

### Downloading a land-cover map for our study area

As we begin to download the data we're going to use, we should remember to record **the Digital Object Identifier (DOI)** for each dataset. DOIs not only allow us to properly cite the dataset and give its authors due credit; they allow others to accurately identify the dataset we used. For the MODIS MCD12Q1 land-cover data, the DOI is:

- DOI: https://doi.org/10.5067/MODIS/MCD12Q1.061

In [ ]:
# Note that TIME_PERIOD refers to one of our global variables, defined at the top of the script
results = earthaccess.search_data(
    short_name = 'MCD12Q1',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

There are two results because our `TIME_PERIOD` spans two years.

In [ ]:
len(results)

As we start to think about a reproducible data-processing pipeline, one thing we might want to avoid is having to re-download large datasets. If the data already exist, let's not download the data again. 

Because we were careful to create separate directories for our datasets, a simple check for the existence of files can be used to determine if we need to download any data. Recall that the `glob.glob()` function will list the contents of our directory search, where the `*` character is a wildcard matching any characters in a filename:

In [ ]:
# This is the directory we're searching
print(f'{LC_DIR}/*')

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{LC_DIR}/*')) == 0:
    earthaccess.download(results, LC_DIR)

**We'll use the `py4eos` library, [which we saw earlier in Module 3,](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources) to open the MODIS MCD12Q1 land-cover dataset for 2023.**

In [ ]:
hdf = py4eos.read_file('data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf', platform = 'MODIS')
hdf

Next, we want to get the land-cover dataset as a `rasterio` dataset so that we can easily resample it to 1-km resolution. We're using the `"LC_Type5"` dataset, which is just one of multiple land-cover classifications that are available in MODIS MCD12Q1.

In [ ]:
lc_raster = hdf.to_rasterio('LC_Type5', filename = '', driver = 'MEM')

Let's generate a preview of our land-cover map.

In [ ]:
lc_map = lc_raster.read(1)
pyplot.imshow(lc_map, interpolation = 'nearest', vmax = 11)

### Projecting the land-cover data

Next, we want to project the land-cover data onto a 1-km EASE-Grid 2.0. It can be difficult to calculate the new width and height of a raster dataset after projection, so we use [the `calculate_default_transform()` function](https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html#rasterio.warp.calculate_default_transform) in the `rasterio.warp` module to figure this out.

In `calculate_default_transform()`, we need to tell it the coordinate reference system (CRS) of our dataset, the new CRS we want, the current width and height (given by `lc_raster.shape`), and the spatial bounds of the raster (given by `lc_raster.bounds`).

The output from this function is a 3-element tuple, so we can assign the output to three variables. The last two are the new `width` and `height` of the projected raster. The first output, `new_transform`, contains other important information about the new, projected raster we're going to create.

In [ ]:
# NOTE: 6933 is the EPSG code for the global EASE-Grid 2.0 projection
new_transform, width, height = calculate_default_transform(
    lc_raster.crs, pyproj.CRS(6933), *lc_raster.shape, *lc_raster.bounds)

Now we're ready to project our land-cover raster using the `project()` function from the `rasterio.warp` module. There are a few of things to note about our code here:

- We need to open an output file to write data into; we use `rasterio.open()` for this and give it the file mode `'w+'` so that we can both write and read data.
- The datatype, or `dtype`, we use in `np.uint8` for 8-bit unsigned integer. 
- We used `Resampling.mode`, or majority resampling, because the numbers in our land-cover raster represent land-cover categories and shouldn't be interpolated as if they are regular numbers.
- The output file, `OUTPUT_LC_FILENAME`, defined at the top of this notebook, will be a GeoTIFF because we gave if the `.tiff` file extension.

In [ ]:
# See more at: https://rasterio.readthedocs.io/en/stable/topics/resampling.html

output_raster = rasterio.open(
    OUTPUT_LC_FILENAME, 'w+', count = 1, width = width, height = height,
    dtype = np.uint8, crs = pyproj.CRS(6933), transform = new_transform)

# Writing the new array to the file
reproject(
    source = rasterio.band(lc_raster, 1),
    destination = rasterio.band(output_raster, 1),
    resampling = Resampling.mode,
    src_nodata = 255,
    dst_nodata = 255)

Finally, we convert our land-cover map to a binary croplands map, where 1 indicates croplands are present and 0 otherwise. According to [the MODIS MCD12Q1 User Guide,](https://lpdaac.usgs.gov/documents/1409/MCD12_User_Guide_V61.pdf) in the `"LC_Type5"` classification, croplands are originally represented by the number 7.

In [ ]:
lc_map_1km = output_raster.read(1)

# Create a binary array where Croplands==1, 0 for everything else
croplands = np.where(lc_map_1km == 7, 1, 0)
pyplot.imshow(croplands, interpolation = 'nearest')

Finally, we're going to overwrite the contents of the output GeoTIFF file we just created, replacing the land-cover map with our new Croplands map. We can do this because the necessary data type (and spatial characteristics) of the data are the same.

In [ ]:
output_raster.write(croplands, 1)
output_raster.close()

---

## Our first Snakefile

Now that we've figured out how to process our land-cover data, let's use Snakemake to generate a rule so that it will be easy to do this again.

Let's create a `Snakefile` with the following rule:

```
rule process_land_cover:
    input:
        "data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf"
    output:
        "data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h15v05_2023.tiff"
    script:
        "workflows/process_land_cover.py"
```

This is a high-level description of our land-cover processing task. We can clearly see the input file, the output file, and the Python script that is going to be executed. 

&#x1F449; **Note that the file paths are relative to the location of the Snakefile.**

&#x1F449; Earlier we used the `shell` keyword in a Snakemake rule to execute arbitrary command-line statements. Instead of `shell`, we are now using the `script` keyword to indicate that Snakemake should run a Python script when executing the rule. Snakemake knows the script is a Python script because of the file extension.

#### &#x1F3C1; Challenge: Writing the land-cover processing script

Our Snakemake rule, above, requires that we have a Python script called `process_land_cover.py`. Write this Python script, based on the steps we took above to process the land-cover data. 

**In your Python script, you can access the `input` and `output` files we specified.** These are known as `snakemake.input` and `snakemake.output`. So, for example, in your Python script you can check to see if the land-cover file has already been downloaded by writing:

```python
# Only download the files once; i.e., if we haven't already downloaded any
if os.path.exists(snakemake.input[0]):
    earthaccess.download(results, os.path.dirname(snakemake.input[0]))
```

**Note that we write `snakemake.input[0]` because there can be multiple inputs!** There can also be multiple outputs. So, when we're ready to create the output raster, we'll write:

```python
output_raster = rasterio.open(
    snakemake.output[0], 'w+', count = 1, width = width, height = height,
    dtype = np.uint8, crs = pyproj.CRS(6933), transform = new_transform)
```

[**Compare your version of `process_land_cover.py` to our script.**](https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/process_land_cover.py)

#### &#x1F6A9; <span style="color:red">Pay Attention</red>

`snakemake.input` and `snakemake.output` are *injected* into our Python code when Snakemake executes the script. **Do not add `import snakemake` to your script. This will not do what you think it does and it will cause an error.**

---

## Downloading ET data

[We previously used the Hargreave's equation (Module 2)](https://github.com/OpenClimateScience/M2-Computational-Climate-Science/blob/main/notebooks/04_Processing_Long_Climate_Data_Records.ipynb) to compute PET, but we could also get PET data from NASA's satellite-based ET algorithm, MODIS MOD16, [as we did in Module 3.](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

In this lesson we'll use PET estimates from [the VIIRS VNP16A2 product](https://dx.doi.org/10.5067/VIIRS/VNP16A2GF.002), which uses the MODIS MOD16 algorithm but is based on data from the newer VIIRS sensor. Both MODIS and VIIRS ET data are available in "gap-filled" (GF) versions, so the short name we use for this product is `'VNP16A2GF'`.

In [ ]:
results = earthaccess.search_data(
    short_name = 'VNP16A2GF',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

Like many MODIS and VIIRS data products, VNP16A2 comes in 8-day composites. There are 46 8-day periods in a single year and our study's time period, `TIME_PERIOD`, spans one year. But, when we check the length of `results`, we can see that there are 47 granules. This is because our results include the final 8-day period that starts on the final day of our time series. In general, it's a good idea to check the length of the results from `earthaccess.search_data()`.

In [ ]:
# Check that we're getting the expected number of granules
len(results)

As before, we'll only download the data if they haven't been downloaded before.

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{VNP16_DIR}/*')) == len(results):
    earthaccess.download(results, VNP16_DIR)

And we'll use `py4eos` again to read the file, even though it is an HDF5 file, because we need an easy way to open the data as a raster dataset.

In [ ]:
hdf = py4eos.read_file('data/VNP16A2GF/VNP16A2GF.A2024001.h18v05.002.2025021191652.h5', platform = 'VIIRS')
hdf

The file contains two datasets that we're interested in:

- `PET_500m` refers to the PET data
- `ET_500m` refers to the ET data (which we'll use later)

[Based on the User Guide (accessed here),](https://dx.doi.org/10.5067/VIIRS/VNP16A2GF.002) we know that both datasets have a valid range and are scaled to make the file smaller. We use `numpy.where()` to assign `np.nan` to values outside the valid range and to re-scale the valid values by `0.1`.

In [ ]:
# Values greater than or equal to 32700 are NoData values
pet0 = hdf.get('PET_500m')
pet = np.where(np.abs(et0) >= 32700, np.nan, et0 * 0.1)

In [ ]:
pyplot.imshow(pet, interpolation = 'nearest')
pyplot.colorbar()

---

## Resampling, projecting ET and PET data

As with the land-cover data, we want to project the ET and PET data onto a 1-km EASE-Grid 2.0. This is slightly more complicated for the ET and PET data, however, as there isn't just a single NoData value; rather, all values greater than 32700 should be considered as invalid values. So, we'll need to use the `read()` method of a `rasterio` dataset to first get a `numpy` array, resampled to 1-km, then write that array into a new `rasterio` dataset before projecting.

#### &#x1F3C1; Challenge

**Write a function that resamples and then projects the VIIRS ET (or PET) data onto a 1-km EASE-Grid 2.0.** Use the function signature below to get started.

```python
def reproject_viirs(hdf, field, output_path = '', driver = 'MEM'):
    '''
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.EOSHDF4
        The EOSHDF4 instance connected to the VIIRS ET dataset
    field : str
        The name of the data variable, e.g., "ET_500m"
    output_path : str
        (Optional) The file path, if writing to a file on disk
    driver : str
        (Optional) The driver name, defaults to "MEM"

    Returns
    -------
    rasterio.io.DatasetWriter
    '''
    ...
```

This is a tough problem! See our solution below.

In [ ]:
def reproject_viirs(hdf, field, output_path = '', driver = 'MEM'):
    '''
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.EOSHDF4
        The EOSHDF4 instance connected to the VIIRS ET dataset
    field : str
        The name of the data variable, e.g., "ET_500m"
    output_path : str
        (Optional) The file path, if writing to a file on disk
    driver : str
        (Optional) The driver name, defaults to "MEM"

    Returns
    -------
    rasterio.io.DatasetWriter
    '''
    et_raster = hdf.to_rasterio(
        field, filename = '', driver = 'MEM', nodata = 32766., scale_and_offset = True)
    
    # First, resample the ET data to 1-km resolution
    arr = et_raster.read(out_shape = (1200, 1200), resampling = Resampling.average)
    arr = np.where(np.abs(arr) >= 32700, np.nan, arr)
    # We have to re-create the raster dataset, now at 1-km resolution
    et_raster_1km = rasterio.open(
        '', 'w+', driver = 'MEM', height = 1200, width = 1200,
        count = 1, dtype = np.float32, crs = et_raster.crs, 
        transform = et_raster.transform * et_raster.transform.scale(2)) # NOTE: Scaling to 1 km
    et_raster_1km.write(arr[0], 1)
    
    # Second, project the data onto a global EASE-Grid 2.0
    new_transform, width, height = calculate_default_transform(
        et_raster_1km.crs, pyproj.CRS(6933), 1200, 1200, *et_raster_1km.bounds)
    et_raster_ease2 = rasterio.open(
        output_path, 'w+', driver = driver, height = height, width = width,
        count = 1, dtype = np.float32, crs = pyproj.CRS(6933), transform = new_transform)
    reproject(
        source = rasterio.band(et_raster_1km, 1),
        destination = rasterio.band(et_raster_ease2, 1),
        resampling = Resampling.bilinear,
        src_nodata = np.nan, # Necessary so that missing data is interpolated
        dst_nodata = np.nan)
    return et_raster_ease2

Let's preview the ET data from the file we opened up earlier.

In [ ]:
et_raster_ease2 = reproject_viirs(hdf, 'ET_500m')
img = et_raster_ease2.read(1)
pyplot.imshow(img, interpolation = 'nearest')
pyplot.colorbar()

Looks good! Now we just need to apply this function to every file we downloaded. We can do that with a `for` loop. We use the `tqdm()` function, from the `tqdm` package, to provide a progress bar.

In [ ]:
from tqdm import tqdm

# Get a list of all the files
file_list = glob.glob(f'{VNP16_DIR}/*')
file_list.sort()

for filename in tqdm(file_list):
    # Read the date from the filename
    date = datetime.datetime.strptime(filename.split('/')[-1].split('.')[1], 'A%Y%j')
    # Convert the date to a YYYYMMDD string
    date_str = date.strftime('%Y%m%d')
    # Prepare the output filename
    output_file_tpl = f'{OUTPUT_VNP16_DIR}/VNP16_%s_mm_8day-1_{date_str}.tiff'
    hdf = py4eos.read_file(filename, platform = 'VIIRS')
    et = reproject_viirs(hdf, 'ET_500m', output_file_tpl % 'ET', driver = 'GTiff')
    pet = reproject_viirs(hdf, 'PET_500m', output_file_tpl % 'PET', driver = 'GTiff')

#### &#x1F3AF; Best Practice

Note that we called `file_list.sort()` above. Why?

It doesn't really matter in this particular case but, in general, **it is often very important to ensure that a list of files is in a certain order before we start processing them.** If the files represent a time series, it may be important to process them in chronological order. Assuming the files have a date string formatted in Year-Month-Day order (e.g., `YYYYMMDD`), another best practice, then a simple call to the file list's `sort()` method will ensure that they are in chronological order.

**To emphasize: When dates in filenames are formatter in Year-Month-Day order (e.g., `YYYYMMDD`), then alphanumeric sorting with the `sort()` function is the same as chronological sorting!**

## TODO Adding ET data processing to SnakeMake

- Generalizing input/ output files using `{templates}`
- Snakemake will only propose to create output files that already exist
- Use of `{wildcards}` in Python commands

---

## Getting precipitation data from IMERG

Next, let's download [precipitation data from IMERG-Final (link)](https://dx.doi.org/10.5067/GPM/IMERGDF/DAY/07).

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDF',
    temporal = TIME_PERIOD)

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{IMERG_DIR}/*')) == 0:
    earthaccess.download(results, IMERG_DIR)

The IMERG-Final data are in netCDF4 format which, [as we discussed in Module 2,](https://github.com/OpenClimateScience/M2-Computational-Climate-Science) can be handled more easily in Python using `xarray`.

The IMERG-Final data also differ from the MODIS land-cover and VIIRS ET data in that each file represents the entire globe. We'll need to crop each file to our study area. An easy way to do this is to use the `bounds` attribute of our `rasterio` dataset, from earlier. We use `shapely.geometry.Polygon` to represent this bounding box in a way that will be understood by the `rioxarray` package. Recall that `rioxarray` is a package built on top of `xarray` that makes it easier to work with spatial datasets in `xarray`.

In [ ]:
# TODO Getting the bounds of our VIIRS tile, for clipping other datasets

bb = et_raster_ease2.bounds
bounds = Polygon([
    (bb.left, bb.bottom), 
    (bb.left, bb.top),
    (bb.right, bb.top),
    (bb.right, bb.bottom)
])
bounds

Note that the coordinates of our bounding box are in meters easting or meters northing (see below). This is because the coordinate reference system (CRS) of our ET data is now a global EASE-Grid 2.0. We'll need to make sure our precipitation data are also projected onto this same CRS.

In [ ]:
bb

Remember that when we have `rioxarray` installed, we can access the `.rio` attribute of any `xarray` dataset to use functions that operate on spatial data. For example...

In [ ]:
# Just an example
ds = xr.open_dataset('data/IMERG/3B-DAY.MS.MRG.3IMERG.20231001-S000000-E235959.V07B.nc4')
ds.rio

We're now ready to process the precipitation data from IMERG-Final. Because each IMERG-Final dataset can be opened as an `xarray.Dataset`, our processing takes the form of a series of steps, each one executed by calling a method on the `xarray.Dataset`.

1. First, we need to get the dimensions of the `precipitation` DataArray ordered by time, latitude, and longitude. We can do this by calling [the `transpose()` method of a Dataset.](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.transpose.html)
2. Next, we need to assign a coordinate reference system (CRS) and tell `xarray` which dimensions correspond to the X and Y coordinate axes of the CRS. We do this with `write_crs()` and `set_spatial_dims()`, respectively. Note that we have to add `.rio` to the beginning of each of these because these functions are part of `rioxarray`.
3. Then, we're ready to project the data to a global EASE-Grid 2.0 (EPSG code 6933). For file size considerations, we'll resample them at 9-km resolution. This is done with the `reproject()` method, another `rioxarray` function.
4. Finally, we're ready to clip the projected data to the bounds (`bounds`) of our study area using the `clip()` method, another `rioxarray` function.

Note that we wrote a `for` loop again, to process each file. We place each projected and clipped dataset into a Python list to store it.

In [ ]:
# NOTE: This may take up to 10 minutes to complete for 1 year's worth of data

file_list = glob.glob(f'{IMERG_DIR}/*.nc4')
file_list.sort()

stack = []
for filename in tqdm(file_list):
    ds = xr.open_dataset(filename)
    # Anytime we use the .rio attribute, we must have rioxarray installed
    ds_ease2 = ds[['precipitation']]\
        .transpose('time', 'lat', 'lon')\
        .rio.write_crs(4326)\
        .rio.set_spatial_dims('lon', 'lat')\
        .rio.reproject(pyproj.CRS(6933), resolution = 9000)\
        .rio.clip([bounds])
    stack.append(ds_ease2)

#### &#x1F3AF; Best Practice

Note that, again, we called `sort()` on the file list, `file_list`. Here, this is essential, because we want the datasets that are put into `stack` to be in chronological order!

**As we've seen before, we can combine all of these separate datasets into a single `xarray.Dataset` using the `concat()` method.**

In [ ]:
ds_precip = xr.concat(stack, dim = 'time')

---

## Packaging derived data products

`ds_precip` only exists in our computer's memory. Let's not forget to write the dataset to disk! This is another things that's easy to do with `xarray` and, if we write the data to a netCDF4 file, all of the attributes and other metadata will be automatically saved as well!

In [ ]:
ds_precip.to_netcdf(f'{OUTPUT_IMERG_DIR}/IMERG_precip_mm_day-1_for_study_area.nc4')

I chose to put the units of precipitation (mm per hour) in our filename but, if I had forgotten to do that, note that it is stored as an attribute of the `precipitation` data.

In [ ]:
ds_precip.precipitation.units

## TODO Adding precip data processing to SnakeMake